# DAKTARI- THE AI MEDICAL CHATBOT

# PROJECT SUMMARY

# 1. BUSINESS UNDERSTANDING

# 2. BUSINESS PROBLEM

# 3. OBJECTIVES

## 3.1 Main objective

## 3.2 Specific objective

## 3.3 Research Questions

## 3.4 Metric of success

# 4. DATA UNDERSTANDING

## 4.1 Data Limitation

# 5. DATA EXPLORATION

## 5.1 Loading a Dataset

In [14]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [15]:
# pip install huggingface_hub


In [16]:
df = pd.read_parquet("hf://datasets/DrBenjamin/ai-medical-chatbot/dialogues.parquet")
df.head()

,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...


In [17]:
df.shape

(256916, 3)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256916 entries, 0 to 256915
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Description  256916 non-null  object
 1   Patient      256916 non-null  object
 2   Doctor       256916 non-null  object
dtypes: object(3)
memory usage: 5.9+ MB


In [19]:
df.isnull().sum()

Description    0
Patient        0
Doctor         0
dtype: int64

Since our data is not missing any missing values we can move to cleaning the text.

## 5.2 Data Cleaning

In [22]:
import nltk # natural language toolkit
import re # regular expressions
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import FreqDist
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
# Download necessary resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Halla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Halla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Halla\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Halla\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### 5.2.1 Text preprocessing

In [25]:
df.columns

Index(['Description', 'Patient', 'Doctor'], dtype='object')

**Tokenization**

Let's breaks down each text into a list of words **tokens**

In [28]:
# Apply tokenization to each text column
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
for col in ['Description', 'Patient', 'Doctor']:
    df[col + '_cleaned'] = df[col].astype(str).apply(word_tokenize)

df[['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']].head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Halla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Halla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,Description_cleaned,Patient_cleaned,Doctor_cleaned
0,"[Q, ., What, does, abutment, of, the, nerve, r...","[Hi, doctor, ,, I, am, just, wondering, what, ...","[Hi, ., I, have, gone, through, your, query, w..."
1,"[Q, ., What, should, I, do, to, reduce, my, we...","[Hi, doctor, ,, I, am, a, 22-year-old, female,...","[Hi, ., You, have, really, done, well, with, t..."
2,"[Q., I, have, started, to, get, lots, of, acne...","[Hi, doctor, !, I, used, to, have, clear, skin...","[Hi, there, Acne, has, multifactorial, etiolog..."
3,"[Q, ., Why, do, I, have, uncomfortable, feelin...","[Hello, doctor, ,, I, am, having, an, uncomfor...","[Hello, ., The, popping, and, discomfort, what..."
4,"[Q, ., My, symptoms, after, intercourse, threa...","[Hello, doctor, ,, Before, two, years, had, se...","[Hello, ., The, HIV, test, uses, a, finger, pr..."


**Lowercasing**

Let's convert our tokenized text to **lowercase** to ensure consistency in our data.

In [31]:
# Convert all tokens to lowercase
for col in ['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']:
    df[col] = df[col].apply(lambda x: [word.lower() for word in x])

df[['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']].head()


,Description_cleaned,Patient_cleaned,Doctor_cleaned
0,"[q, ., what, does, abutment, of, the, nerve, r...","[hi, doctor, ,, i, am, just, wondering, what, ...","[hi, ., i, have, gone, through, your, query, w..."
1,"[q, ., what, should, i, do, to, reduce, my, we...","[hi, doctor, ,, i, am, a, 22-year-old, female,...","[hi, ., you, have, really, done, well, with, t..."
2,"[q., i, have, started, to, get, lots, of, acne...","[hi, doctor, !, i, used, to, have, clear, skin...","[hi, there, acne, has, multifactorial, etiolog..."
3,"[q, ., why, do, i, have, uncomfortable, feelin...","[hello, doctor, ,, i, am, having, an, uncomfor...","[hello, ., the, popping, and, discomfort, what..."
4,"[q, ., my, symptoms, after, intercourse, threa...","[hello, doctor, ,, before, two, years, had, se...","[hello, ., the, hiv, test, uses, a, finger, pr..."


**Stopword Removal**

By removing **Stopwords** like *the, is, and*   which don’t really  carry meaningful information we reduce the noise in our data.

In [34]:
words_to_remove = ['hi', 'hello', 'doctor', 'thanks', 'thank', 'please', 'kindly','q','but']
stop_words = set(stopwords.words('english')+ words_to_remove)

# Remove stopwords
for col in ['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']:
    df[col] = df[col].apply(lambda x: [word for word in x if word not in stop_words])

df[['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']].head()

,Description_cleaned,Patient_cleaned,Doctor_cleaned
0,"[., abutment, nerve, root, mean, ?]","[,, wondering, abutting, abutment, nerve, root...","[., gone, query, diligence, would, like, know,..."
1,"[., reduce, weight, gained, due, genetic, hypo...","[,, 22-year-old, female, diagnosed, hypothyroi...","[., really, done, well, hypothyroidism, proble..."
2,"[q., started, get, lots, acne, face, ,, partic...","[!, used, clear, skin, since, moved, new, plac...","[acne, multifactorial, etiology, ., acne, soap..."
3,"[., uncomfortable, feeling, middle, spine, lef...","[,, uncomfortable, feeling, middle, spine, lef...","[., popping, discomfort, felt, either, imprope..."
4,"[., symptoms, intercourse, threatns, even, neg...","[,, two, years, sex, call, girl, dark, locatio...","[., hiv, test, uses, finger, prick, blood, sam..."


**Punctuation Removal**

Removing punctuation marks like *!, ., ,*  because they rarely add semantic meaning in our text analyses.

In [37]:
# Remove punctuation
for col in ['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']:
    df[col] = df[col].apply(lambda x: [word for word in x if word not in string.punctuation])

df[['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']].head()


,Description_cleaned,Patient_cleaned,Doctor_cleaned
0,"[abutment, nerve, root, mean]","[wondering, abutting, abutment, nerve, root, m...","[gone, query, diligence, would, like, know, he..."
1,"[reduce, weight, gained, due, genetic, hypothy...","[22-year-old, female, diagnosed, hypothyroidis...","[really, done, well, hypothyroidism, problem, ..."
2,"[q., started, get, lots, acne, face, particula...","[used, clear, skin, since, moved, new, place, ...","[acne, multifactorial, etiology, acne, soap, i..."
3,"[uncomfortable, feeling, middle, spine, left, ...","[uncomfortable, feeling, middle, spine, left, ...","[popping, discomfort, felt, either, improper, ..."
4,"[symptoms, intercourse, threatns, even, negati...","[two, years, sex, call, girl, dark, location, ...","[hiv, test, uses, finger, prick, blood, sample..."


**POS Tagging and Lemmatization**

**POS Tagging** involves labeling each word in a sentence with its grammatical part of speech such as nouns, adjectives, verbs and adjectives

**Lemmatization** refines this using linguistic context better to good.
Both this steps will help us unify variations of the same word.

In [40]:
#POS tagging
def get_pos(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,  # Represents an Adjective
        'N': wordnet.NOUN, # Represents a Noun
        'V': wordnet.VERB, # Represents a Verb
        'R': wordnet.ADV   # Represents an Adverb
    }
    return tag_dict.get(tag, wordnet.NOUN)

In [41]:
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Halla\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Halla\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

# Apply to each column
for col in ['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']:
    df[col] = df[col].apply(lemmatize_tokens)

df[['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']].head()

In [ ]:
!pip install -U sentence-transformers

In [ ]:
pip install tf-keras

In [ ]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for the combined text
df['combined_text'] = df['Description_cleaned'].apply(lambda x: ' '.join(x)) + ' ' + \
                      df['Patient_cleaned'].apply(lambda x: ' '.join(x)) + ' ' + \
                      df['Doctor_cleaned'].apply(lambda x: ' '.join(x))

# Generate embeddings
embeddings = model.encode(df['combined_text'], show_progress_bar=True)

# embeddings is a NumPy array you can use for clustering, similarity, or ML
embeddings.shape


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine relevant columns into one text field
df['combined_text'] = df['Description_cleaned'].apply(lambda x: ' '.join(x)) + ' ' + \
                      df['Patient_cleaned'].apply(lambda x: ' '.join(x)) + ' ' + \
                      df['Doctor_cleaned'].apply(lambda x: ' '.join(x))

# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(
     max_features=1000,   # instead of 5000
    stop_words='english',
    ngram_range=(1,2)
)

# Fit and transform
X_tfidf = tfidf.fit_transform(df['combined_text'])

print(X_tfidf.shape)

**Text Normalization**

Finally, we normalize the text by joining the processed tokens back into sentences.

In [ ]:
# Convert token lists to cleaned strings
for col in ['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']:
    df[col.replace('_tokens', '_cleaned')] = df[col].apply(lambda x: ' '.join(x))

# Display the first few rows
df[['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']].head()


In [ ]:
### **Exploratory Data Analysis (EDA)**
# Word Frequency Distribution for Patient messages
all_patient_words = [word for tokens in df['Patient_cleaned'] for word in tokens]
freq_dist_patient = FreqDist(all_patient_words)
most_common_patient = freq_dist_patient.most_common(20)
# Plotting the most common words
words, counts = zip(*most_common_patient)
plt.figure(figsize=(12, 6))
plt.bar(words, counts)
plt.title('Most Common Words in Patient Messages')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()



In [ ]:
df.columns

In [ ]:
# Word Frequency Distribution for Doctor messages
all_doctor_words = [word for tokens in df['Doctor_cleaned'] for word in tokens]
freq_dist_doctor = FreqDist(all_doctor_words)
most_common_doctor = freq_dist_doctor.most_common(20)
# Plotting the most common words
words, counts = zip(*most_common_doctor)
plt.figure(figsize=(12, 6))
plt.bar(words, counts)
plt.title('Most Common Words in Doctor Messages')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

In [ ]:

### **Visualization of Message Lengths**
# Calculate message lengths
df['Patient_msg_length'] = df['Patient_cleaned'].apply(len)
df['Doctor_msg_length'] = df['Doctor_cleaned'].apply(len)

# Plotting the message lengths
plt.figure(figsize=(12, 6))
plt.hist(df['Patient_msg_length'], bins=30, alpha=0.5, label='Patient Messages')
plt.hist(df['Doctor_msg_length'], bins=30, alpha=0.5, label='Doctor Messages')
plt.title('Distribution of Message Lengths')
plt.xlabel('Message Length')
plt.ylabel('Frequency')
plt.legend()
plt.show()


In [ ]:
# Create new columns for text length in words
for col in ['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']:
    df[col + '_length'] = df[col].apply(lambda x: len(str(x).split()))

# Plot word count distribution for each text column
for col in ['Description_cleaned', 'Patient_cleaned', 'Doctor_cleaned']:
    plt.figure(figsize=(6,4))
    sns.histplot(df[col + '_length'], bins=30, kde=True, color='teal')
    plt.title(f"Word Count Distribution in {col}")
    plt.xlabel("Word Count")
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
# ploting common pair of words in patient and doctor messages
from nltk import bigrams
from collections import Counter
patient_bigrams = [bigram for tokens in df['Patient_cleaned'] for bigram in bigrams(tokens)]
doctor_bigrams = [bigram for tokens in df['Doctor_cleaned'] for bigram in bigrams(tokens)]
patient_bigram_counts = Counter(patient_bigrams).most_common(20)
doctor_bigram_counts = Counter(doctor_bigrams).most_common(20)
# Plotting Patient Bigrams
patient_bigrams_words, patient_bigrams_counts = zip(*patient_bigram_counts)
plt.figure(figsize=(12, 6))
plt.bar([' '.join(bigram) for bigram in patient_bigrams_words], patient_bigrams_counts)
plt.title('Most Common Bigrams in Patient Messages')
plt.xlabel('Bigrams')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

In [ ]:
!pip install wordcloud

In [ ]:
print(type(df['Patient_cleaned'].iloc[0]))

In [ ]:
# ploting wordcloud for doctor and patient messages
from wordcloud import WordCloud
# Generate wordcloud for Patient messages
patient_text = ' '.join(df['Patient_cleaned'])
patient_wordcloud = WordCloud(width=800, height=400, background_color='white').generate(patient_text)
# Plotting the wordcloud
plt.figure(figsize=(10, 5))
plt.imshow(patient_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Patient Messages')
plt.show()

## Feature Extraction

In [ ]:
#  FEATURE EXTRACTION using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['combined_text'])

print("Feature matrix shape:", X.shape)